In [493]:
import pandas as pd
import re
import pymorphy2
from sklearn.model_selection import train_test_split

In [494]:
df = pd.read_csv('train.csv')

In [495]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#df = df.loc[(df['1category'] != '?') & (df['2category'].isna())]


In [496]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [497]:
result = " ".join(df.where(df['sentiment'] == "+").dropna(how = 'all')['sentence'])

from collections import Counter

words = re.findall(r'\b\w+\b', result.lower())
words = [word for word in words if word not in stopwords]
words = [morph.parse(word)[0].normal_form for word in words]
good = Counter(words)


result = " ".join(df.where(df['sentiment'] == "−").dropna(how = 'all')['sentence'])
words = re.findall(r'\b\w+\b', result.lower())
words = [word for word in words if word not in stopwords]
words = [morph.parse(word)[0].normal_form for word in words]
bad = Counter(words)

bad = bad.most_common(100)


good = good.most_common(100)

good_list = []
bad_list = []
for i in range(100):
    good_list.append(good[i][0])
    bad_list.append(bad[i][0])


# Преобразование списков в множества
set1 = set(good_list)
set2 = set(bad_list)

# Получение уникальных элементов каждого множества
unique_set1 = set1.difference(set2)
unique_set2 = set2.difference(set1)

# Преобразование множеств обратно в списки
only_good = list(unique_set1)
only_bad = list(unique_set2)

print(only_good) 
print(only_bad)  


['уровень', 'помощь', 'выразить', 'объяснить', 'профессионализм', 'довольный', 'общение', 'грамотный', 'оперативность', 'тинькофф', 'удобный', 'огромный', 'положительный', 'быстро', 'вклад', 'приятно', 'отличный', 'помочь', 'высокий', 'рекомендовать', 'вежливый', 'ответить', 'оперативно', 'отметить', 'подробно', 'условие', 'спасибо', 'понравиться', 'плюс', 'благодарность', 'решение', 'всё', 'приятный', 'продукт', 'приложение', 'наш', 'удобно', 'девушка', 'также', 'дело', 'профессиональный', 'впечатление', 'хороший', 'платёж', 'большой', 'любой', 'интернет', 'рассказать']
['1', 'считать', 'обращение', 'срок', 'писать', '3', '2', 'говорить', 'деньга', 'оператор', 'один', 'должный', 'крайне', 'некомпетентный', 'звонить', 'информация', 'отвратительный', 'линия', 'абсолютно', 'звонок', 'советовать', 'месяц', 'знать', 'никто', 'сумма', 'заявление', 'позвонить', 'ужасный', 'сегодня', 'вообще', 'написать', 'заявка', 'ещё', 'горячий', 'договор', 'банкомат', 'полный', 'ответ', 'никакой', 'ситуац

In [498]:
df['num_digits'] = df['sentence'].str.count(r'\d')

# Функция, которая проверяет наличие даты в формате xx.xx.xxxx в строке
def has_date(date_string):
    patterns = [
        r'\b\d{2}\.\d{2}\.\d{4}\b',    # Формат xx.xx.xxxx
        r'\b\d{2}[-/]\d{2}[-/]\d{2}\b', # Формат xx-xx-xx или xx/xx/xx
        # Дополнительные паттерны, включая варианты с названиями месяцев на русском языке
        r'\b\d{1,2} [Яя]нвар[яь] \d{4}\b',
        r'\b\d{1,2} [Фф]еврал[яь] \d{4}\b',
        r'\b\d{1,2} [Мм]арт[а] \d{4}\b',
        r'\b\d{1,2} [Аа]прел[яь] \d{4}\b',
        r'\b\d{1,2} [Мм]а[яй] \d{4}\b',
        r'\b\d{1,2} [Ии]юн[яь] \d{4}\b',
        r'\b\d{1,2} [Ии]юл[яь] \d{4}\b',
        r'\b\d{1,2} [Аа]вгуст[а] \d{4}\b',
        r'\b\d{1,2} [Сс]ентябр[яь] \d{4}\b',
        r'\b\d{1,2} [Оо]ктябр[яь] \d{4}\b',
        r'\b\d{1,2} [Нн]оябр[яь] \d{4}\b',
        r'\b\d{1,2} [Дд]екабр[яь] \d{4}\b',
    ]
    for pattern in patterns:
        match = re.search(pattern, date_string)
        if match:
            return 1
    return 0
df['has_date'] = df['sentence'].apply(has_date)

# Функция, которая проверяет наличие даты в формате xx.xx.xxxx в строке
def has_ad(date_string):
    address_patterns = [r'([А-ЯЁ][а-яё]+(?:-+[А-ЯЁ][а-яё]+)*(?: [0-9]+[а-яё]*)(?:/[0-9]+)*)',
                        r'([А-ЯЁ][а-яё]+(?:-+[А-ЯЁ][а-яё]+)*)[, ]+([А-ЯЁ][а-яё]+(?:-+[А-ЯЁ][а-яё]+)*(?: [0-9]+[а-яё]*)(?:/[0-9]+)*)'
                        ,r'([А-ЯЁ][а-яё]+(?:-+[А-ЯЁ][а-яё]+)*(?: [0-9]+[а-яё]*)(?:/[0-9]+)*)(?:,| в)+([А-ЯЁ][а-яё]+(?:-+[А-ЯЁ][а-яё]+)*)']
    for pattern in address_patterns:
        match = re.search(pattern, date_string)
        if match is not None:
            return 1
    return 0

# Добавляем новый столбец 'has_ad'
df['has_ad'] = df['sentence'].apply(has_ad)

def has_phone_number (date_string):
    patterns = [
    r"\+7\d{10}",
    r"8\d{10}",
    r"8\s?\d{3}\s?\d{3}-?\d{2}-?\d{2}",
    r"\d{11}"
    ]
    for pattern in patterns:
        match = re.search(pattern, date_string)
        if match is not None:
            return 1
    return 0
df['has_phone_number'] = df['sentence'].apply(has_phone_number)

def len_review (date_string):
    return len(date_string)
df['len_review'] = df['sentence'].apply(len_review)

def warning_sign1(date_string):
    if '!' in date_string:
        return 1
    else: 
        return 0
df['warning_sign1'] = df['sentence'].apply(warning_sign1)    

def count_warning_signs(s):
    return s.count("!")
df['count_warning_signs'] = df['sentence'].apply(count_warning_signs)

def caps_words_count(df):
    caps_words_count = []
    for sentence in df['sentence']:
        caps_words = [word for word in sentence.split() if word.isupper()]
        caps_words_count.append(len(caps_words))
    df['caps_words_count'] = caps_words_count
    return df
df = caps_words_count(df)

def get_bracket_value(text):
    count_open = text.count("(")
    count_close = text.count(")")
    if count_open > count_close:
        return 1
    elif count_open < count_close:
        return 2
    else:
        return 0
df['brackets'] = df['sentence'].apply(get_bracket_value)

def has_english_letters(s):
    """
    Функция принимает строку s и возвращает 1, если в ней есть английские буквы,
    и возвращает 0 в противном случае.
    """
    for char in s:
        if char.isalpha() and char.isascii():
            # Проверяем, является ли символ буквой и английским символом 
            return 1
    return 0
df['has_english_letters'] = df['sentence'].apply(has_english_letters)

def count_answer_sign(s):
    return s.count("!")
df['count_answer_sign'] = df['sentence'].apply(count_answer_sign)

def count_dots(s):
    return s.count(".")
df['count_dots'] = df['sentence'].apply(count_dots)

def count_minus(s):
    return s.count("-")
df['count_minus'] = df['sentence'].apply(count_minus)

def count_minus(s):
    return s.count("-")
df['count_minus'] = df['sentence'].apply(count_minus)

def count_plus(s):
    return s.count("+")
df['count_plus'] = df['sentence'].apply(count_plus)

def гub(date_string):
    if 'руб' in date_string or 'тыс' in date_string:
        return 1
    else: 
        return 0
df['гub'] = df['sentence'].apply(гub)

In [499]:
def mood(df):
    good_words_count = []
    bad_words_count = []
    words = re.findall(r'\b\w+\b', result.lower())
    words = [word for word in words if word not in stopwords]
    words = [morph.parse(word)[0].normal_form for word in words]
    for sentence in df['sentence']:
        words = re.findall(r'\b\w+\b', sentence.lower())
        words = [word for word in words if word not in stopwords]
        words = [morph.parse(word)[0].normal_form for word in words]
        good_counts = {word: words.count(word) for word in only_good}
        bad_counts = {word: words.count(word) for word in only_bad}
        good_words_count.append(good_counts)
        bad_words_count.append(bad_counts)
    df['good_words'] = good_words_count
    df['bad_words'] = bad_words_count
    return df
df = mood(df)
    

In [500]:
df['good_c'] = 0
df['bad_c'] = 0
for i in range(len(df.index)):
    
    df['good_c'][df.index[i]] = sum(df['good_words'][df.index[i]].values())
    df['bad_c'][df.index[i]] = sum(df['bad_words'][df.index[i]].values())

<ipython-input-8-88568bb7e1f3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['good_c'][df.index[i]] = sum(df['good_words'][df.index[i]].values())
<ipython-input-8-88568bb7e1f3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bad_c'][df.index[i]] = sum(df['bad_words'][df.index[i]].values())


In [501]:

banks=['Сбербанк','ВТБ','Альфа','Тинькофф','Сити','Россельхозбанк','Росбанк','Райф','МТС','ИКС','Уралсиб','Банк России','Банк Открытие'] 
import spacy
import numpy as np
nlp=spacy.blank('ru')

def find_bank_names(docs):
    bank_names = []
    for doc in docs:
        found = False
        for token in doc:
            if token.text in banks:
                found = True
                bank_names.append(token.text)
                break
        if not found:
            bank_names.append(None)
    return bank_names

def mark_bank_name(df):
    docs = list(nlp.pipe(df['sentence']))
    df['bank_name'] = find_bank_names(docs) 
    return df 
df=mark_bank_name(df)
groupby_bank = df.groupby('bank_name')[['1category', '2category','sentiment']].agg(lambda x: x.mode().iloc[0]).reset_index()
merged = pd.merge(df, groupby_bank, on='bank_name', how='left')
for bank in banks:
    col_name = f"{bank}_1category_mode"
    merged[col_name] = merged['1category_y'].where(merged['bank_name'] == bank, np.nan)
    col_name = f"{bank}_2category_mode"
    merged[col_name] = merged['2category_y'].where(merged['bank_name'] == bank, np.nan)
    col_name = f"{bank}_sentiment_mode"
    merged[col_name] = merged['sentiment_y'].where(merged['bank_name'] == bank, np.nan)
merged=merged.drop(columns=(['1category_y','2category_y','sentiment_y']))


In [502]:
bank_map = {bank_name: i for i, bank_name in enumerate(banks)}
df['bank_name'] = df['bank_name'].map(bank_map)
df['bank_name'].fillna(0, inplace= True)


In [ ]:
df['bank_name'].unique()

In [ ]:
df

In [505]:
from sklearn import tree
import matplotlib.pyplot as plt


from IPython.display import SVG
from graphviz import Source
from IPython.display import display

import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import mean_squared_error,r2_score 

import warnings
warnings.filterwarnings('ignore')
breast_cancer = load_breast_cancer()

import pandas as pd
import numpy as np
import re
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix, precision_score, recall_score

pd.options.display.max_columns = 100

In [506]:
x = df.drop(columns= ['Unnamed: 0', 'sentence', '1category', '2category', 'sentiment', 'good_words', 'bad_words' ])
y = pd.DataFrame(df['sentiment'])
y['sentiment'] = y['sentiment'].replace({'+': 1, '−': 0, '?': 2})
y = y.astype(float)


In [507]:
#!c1.32
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(x, y.values.ravel(), test_size=0.1)

# Создание объекта классификатора RandomForestClassifier
clf = RandomForestClassifier()

# Определение сетки гиперпараметров для GridSearchCV
param_grid = {'n_estimators': [250],
              'max_depth': [20, 30]}

# Создание объекта GridSearchCV
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10, scoring='roc_auc_ovr')

# Обучение классификатора с использованием GridSearchCV
grid_search.fit(X_train, y_train)

# Вычисление ROC AUC для многоклассовой классификации с методом OvR
y_pred_proba = grid_search.predict_proba(X_test)
roc_auc_ovr = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
#train_score = clf.score(X_train, y_train)
#test_score = clf.score(X_test, y_test)
#mean_cross_val_score = cross_val_score(clf, X_train, y_train, cv = 10).mean()
#temp_score_data = pd.DataFrame({"max_depth": [max_depth], "train_score" : [train_score], "test_score": [test_score], "mean_cross_val_score": [mean_cross_val_score]})
#scores_data = scores_data.append(temp_score_data)
# Вывод результатов
print("Best parameters:", grid_search.best_params_)
print("ROC AUC (OvR):", roc_auc_ovr)

Best parameters: {'max_depth': 30, 'n_estimators': 250}
ROC AUC (OvR): 0.9017491023947439


In [ ]:
scores_data_long = pd.melt(scores_data, id_vars = ["max_depth"], value_vars = ["train_score", "test_score", "mean_cross_val_score"], var_name = "set_type", value_name= "score")
sns.lineplot(x = "max_depth", y = "score", hue = "set_type", data = scores_data_long )

In [508]:
from catboost import CatBoostClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [ ]:
from catboost import CatBoostClassifier

# Создание классификатора
clf = CatBoostClassifier(eval_metric='MultiClass')

# Обучение классификатора
clf.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = clf.predict(X_test)

# Вычисление ROC AUC
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr')
print('ROC AUC:', roc_auc)


In [513]:
from sklearn.model_selection import GridSearchCV

# Определение параметров для подбора
params = {
    'iterations': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [10,16]
}

# Создание объекта классификатора
clf = CatBoostClassifier(loss_function='MultiClass')

# Создание объекта GridSearchCV
grid_search = GridSearchCV(clf, params, cv=5, scoring='roc_auc_ovr')

# Обучение классификатора с перебором параметров
grid_search.fit(X_train, y_train)

# Определение лучших параметров
best_params = grid_search.best_params_
print('Лучшие параметры:', best_params)

# Оценка качества модели с лучшими параметрами на тестовой выборке
clf_best = CatBoostClassifier(**best_params, loss_function='MultiClass', eval_metric='MultiClassOneVsAll', random_seed=42)
clf_best.fit(X_train, y_train)
y_pred = clf_best.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
print('ROC AUC (One vs Rest):', roc_auc)


0:	learn: 1.0919148	total: 105ms	remaining: 10.4s
1:	learn: 1.0856356	total: 126ms	remaining: 6.18s
2:	learn: 1.0794835	total: 134ms	remaining: 4.32s
3:	learn: 1.0733048	total: 151ms	remaining: 3.62s
4:	learn: 1.0672526	total: 168ms	remaining: 3.19s
5:	learn: 1.0613161	total: 185ms	remaining: 2.9s
6:	learn: 1.0554837	total: 202ms	remaining: 2.69s
7:	learn: 1.0498051	total: 213ms	remaining: 2.45s
8:	learn: 1.0444118	total: 233ms	remaining: 2.35s
9:	learn: 1.0389104	total: 251ms	remaining: 2.26s
10:	learn: 1.0336334	total: 268ms	remaining: 2.17s
11:	learn: 1.0284659	total: 294ms	remaining: 2.15s
12:	learn: 1.0233889	total: 303ms	remaining: 2.03s
13:	learn: 1.0183818	total: 332ms	remaining: 2.04s
14:	learn: 1.0134309	total: 349ms	remaining: 1.98s
15:	learn: 1.0086443	total: 365ms	remaining: 1.92s
16:	learn: 1.0040088	total: 383ms	remaining: 1.87s
17:	learn: 0.9994710	total: 401ms	remaining: 1.83s
18:	learn: 0.9950385	total: 420ms	remaining: 1.79s
19:	learn: 0.9905759	total: 439ms	remainin

CatBoostError: catboost/private/libs/options/plain_options_helper.cpp:194: MultiClass and MultiClassOneVsAll are incompatible

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=10)
clf.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
print('ROC AUC score:', roc_auc)

In [ ]:
y['sentiment'].unique()

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Загружаем предварительно обученную модель BERT
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

# Загружаем токенизатор
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Задаем текст для анализа
text = "Этот продукт действительно потрясающий! Я советую его всем!"

# Токенизируем текст и добавляем специальные токены
input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])

# Передаем токенизированный текст в модель и получаем логиты
logits = model(input_ids)[0]

# Применяем функцию softmax к логитам и получаем вероятности для каждого класса
probs = torch.nn.functional.softmax(logits, dim=-1)

# Выбираем наиболее вероятный класс
pred_class = torch.argmax(probs, dim=-1).item()

# Выводим результат
if pred_class == 0:
    print("Отзыв отрицательный")
else:
    print("Отзыв положительный")
